Config.py

In [ ]:
@dataclass
class RbcParams:
    """RBC model parameters"""
    
    α:     float = 0.36     # Capital share
    η:     float = 0.34     # Labor weight
    ρ:     float = 0.918    # TFP shock persistence
    β:     float = 0.96     # Discount factor
    δ:     float = 0.1      # Depreciation rate
    μ_ϵ:   float = 0.0      # Mean of TFP
    σ_ϵ:   float = 0.014    # Std of TFP
    

    def __post_init(self):
        """Full depreciation case δ=1.0, closed-form solution"""
        self.γ = ((1.0 - self.α)*self.η)/((1.0 - self.α*self.β)*(1 - self.η))

        # Labor supply when δ=1.0
        self.h_bar = self.γ/(1 + self.γ)

    def prod_fn(self, a_t, k_t, h_t):
        """Cobb-Douglas production function"""
        return a_t * (k_t ** self.α) * (h_t ** (1-self.α))

    
    def consumption_full_dep(self, y_t):
        """Closed-form solution for consumption under full depreciation"""
        return (1 - self.α * self.β) * y_t
    
    def Ξ_full_dep(self, neural_net):
        """Loss function under full depreciation"""
        
        

In [ ]:
import torch
import torch.nn as nn
from dataclasses import dataclass, field
from typing import List, Union, Optional


@dataclass
class NetworkParams:
    """ Neural net parameters"""

    in_dim:  int = 2        # input dimension (k_t, a_t)
    hid_dim: List[int] = field(default_factory=lambda: [16]) # Hidden layer
    out_dim: int = 1  # output dimension (consumption share)
    act_fn: Union[str, List[str]] = "relu"  # Activation function(s)
    dropout_rate: float = 0.0 # Dropout rate
    num_batch: int = 100   # Batch size
    num_epoch: int = 5000   # Epoch size
    optimizer: str = "adam"
    out_bound: float = 1e-6  # output bound
    lr: float = 1e-3 # learning rate
    freq_gamma: float = 0.85    # Exponential Decay rate for Exponential Scheduler
    LRschedule: str = "exponential"   # Learning rate scheduler
    
    def __post_init__(self):
        """Validate activation functions, optimizers and scheduler"""

        # Valid activtion functions
        valid_activations = ["relu", "tanh", "sigmoid"]

        if isinstance(self.act_fn, str):
            if self.act_fn.lower() not in valid_activations:
                raise ValueError(f"Activation function must be one of: {valid_activations}")
        else:
            for act in self.act_fn:
                if act.lower() not in valid_activations:
                    raise ValueError(f"Activation function must be one of: {valid_activations}")

        # Valid optimizers
        valid_optimizers = ["adam", "sgd"]
        if self.optimizer.lower() not in valid_optimizers:
            raise ValueError(f"Optimizer must be one of: {valid_optimizers}")

        valid_schedulers = ["exponential"]
        if self.LRscheduler.lower() not in valid_schedulers:
            raise ValueError(f"Learning rate scheduler must be one of {valid_schedulers}")
        
    def get_act_fn(self, idx=0) -> nn.Module:
        """Convert activation string to PyTorch activation function"""
        if isinstance(self.act_fn, list):
            if idx >= len(self.act_fn):
                act_name = self.act_fn[-1]
            else:
                act_name = self.act_fn[idx]
        else:
            act_name = self.act_fn

        act_name = act_name.lower()
        if act_name == "relu":
            return nn.ReLU()
        elif act_name == "tanh":
            return nn.Tanh()
        elif act_name == "sigmoid":
            return nn.Sigmoid()
            
    def get_optimizer(self, neural_net) -> torch.optim.Optimizer:
        """Convert optimizer string to PyTorch Optimizer"""
        opt_name = self.optimizer.lower()
        if opt_name == "adam":
            return torch.optim.Adam(neural_net.parameters(), lr = self.lr)
        if opt_name == "sgd":
            return torch.optim.SGD(neural_net.parameters(), lr= self.lr)

    def get_scheduler(self, optimizer) -> torch.optim.lr_scheduler:
        """Convert LRscheduler string to PyTorch scheduler"""
        scheduler_name = self.LRscheduler.lower()
        if scheduler_name == "exponential":
            return torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = self.freq_gamma)
        

Neural network model

In [ ]:
class NeuralNetwork(nn.Module):
    """Neural Network model with configurable architecture"""
    def __init__(self, params: NetworkParams):
        """Initialize neural network based on provided parameters"""
        super().__init__()
        self.params = params

        # Initialize layers
        layers = []

        # Input layer
        layers.append(nn.Linear(params.in_dim, params.hid_dim[0]))
        if params.dropout_rate > 0:
            layers.append(nn.Dropout(p=params.dropout_rate))
        layers.append(params.get_act_fn(0))

        # Hidden Layers
        for i in range(1, len(params.hid_dim)):
            layers.append(nn.Linear(params.hid_dim[i-1], params.hid_dim[i]))
            if params.dropout_rate > 0:
                layers.append(nn.Dropout(p=params.dropout_rate))
            layers.append(params.get_act_fn(i))

        # Output Layers
        layers.append(nn.Linear(params.hid_dim[-1], params.out_dim))

        # put the layers together
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        """Forward pass through the network"""
 
        out = self.layers(x)

        # Apply sigmoid to output
        ζ_0 = torch.sigmoid(out)

        # bound the output
        ζ_1 = torch.minimum(
            torch.maximum(ζ_0, torch.tensor([self.params.out_bound])),
            torch.tensor([1.0 - self.params.out_bound])
        )

        return ζ_1



    def normalized_forward(self, params: RbcParams, x):
        """Forward pass with normalized input"""

        # Capital input
        k_t = x[:, 0].unsqueeze(1)

        # TFP input
        a_t = x[:, 1].unsqueeze(1)

        # Normalized capital input
        k_t_norm = 
        


    
    def initialization(params: NetworkParams, loss_freq = 1000):
    """Initialize the neural network with full depreciation case as a guess"""

        # Instantiate a neural net
        net = NeuralNetwork(params)

        # Train mode
        net.train()

        # get optimizer
        optimizer = params.get_optimizer(net)

        # get scheduler
        scheduler = params.get_scheduler(optimizer)

        # Initialize epoch size loss tensor
        loss_epoch = torch.zeros(params.num_epoch)
    
        for i in range(0, params.num_epoch):
            optimizer.zero_grad()
            loss 
    

Build loss function for the RBC model